In [3]:
def make_dict(sub):
    check_dict[sub] = { 
            "ANAT_CHECK":None,
            "FMAP_CHECK":None,
            "MILKSHAKES": [],
            "MILK_CHECK": None,
            "IMAGINE_CHECK": None,
            "GO1_CHECK": None,
            "GO2_CHECK": None
    }

In [2]:
import os, shutil, glob
import pandas as pd 

#BIDS_PATH = 
#MULTI_SESS =

#def check_subjects():
## Set input path for BIDS data and get subjects 
session_ID = "ses-2"
SUB_DIRS = glob.glob(os.path.join("/projects/niblab/bids_projects/Experiments/ChocoData/BIDS", session_ID, "sub-*"))
check_dict = {} ## initialize dictionary
## iterate through subjects and inspect their files
for SUBDIR in sorted(SUB_DIRS):
    SUBJECT = SUBDIR.split("/")[-1] # get subject id
    # MAKE DICTIONARY 
    make_dict(SUBJECT) # add subject id to dictionary
    # GET RELEVANT PATHS, AND FILES 
    ## --here we set the data directory paths we want to check
    ## (func/, anat/, and fmap/ path)
    ## and then we grab all the *nii.gz files within them
    funcpath = os.path.join(SUBDIR, 'func', '*nii.gz')
    anatpath = os.path.join(SUBDIR, 'anat', '*nii.gz')
    fmappath = os.path.join(SUBDIR, 'fmap', '*nii.gz')
    FUNCS = glob.glob(funcpath)
    ANATS = glob.glob(anatpath)
    FMAPS = glob.glob(fmappath)
    # IDENTIFY FUNCS AND ANY ERRORS
    if not FUNCS:
        #print("********ERROR! NO FUNCTIONALS: ", SUBJECT)
        func_check = "FAIL"
        #######WRITE TO FILE -- QUALITY ANALYSIS REPORTING
    else: # IDENTIFY FUNCS
        milk_count = 0
        imag_count = 0
        go1_count = 0
        go2_count = 0 
        for func in FUNCS:
            if "_2" in SUBJECT:
                TASK = func.split("_")[5]
                TASK_TYPE = func.split("_")[6].split(".")[0]
            else:
                TASK = func.split("_")[3]
                TASK_TYPE = func.split("_")[4].split(".")[0]
            if "milkshake" in TASK:
                milk_count = milk_count + 1
                curr_mlk_ID = TASK.split("-")[1].split("e")[1]
                #print(curr_mlk_ID)
                check_dict[SUBJECT]["MILKSHAKES"].append(curr_mlk_ID)
                #print("MILKSHAKE", TASK)
            elif "imagine" in TASK:
                imag_count = imag_count + 1
                #check_dict[SUBJECT]["IMAGINE"]
                #print("Imagine: ", TASK)
            else:
                if "GoNoGo1" in TASK:
                    go1_count = go1_count + 1
                    #print("GoNoGo1", TASK)
                else:
                    go2_count = go2_count + 1
                    #print("GoNoGo2: ", TASK  )
        #print(milk_count, imag_count, go1_count, go2_count)
    if milk_count == 4:
        MILK_CHECK = "PASS"
    elif milk_count == 0: 
        MILK_CHECK = "FAIL"
    else:
        MILK_CHECK = "PARTIAL PASS"
    if imag_count == 2:
        IMAGINE_CHECK = "PASS"
    else:
        IMAGINE_CHECK = "FAIL"
    if go1_count == 2:
        GO1_CHECK = "PASS"
    else:
        GO1_CHECK = "FAIL"
    if go2_count == 2:
        GO2_CHECK = "PASS"
    else:
        GO2_CHECK = "FAIL"
            
    check_dict[SUBJECT]["MILK_CHECK"] = MILK_CHECK
    check_dict[SUBJECT]["IMAGINE_CHECK"] = IMAGINE_CHECK
    check_dict[SUBJECT]["GO1_CHECK"] = GO1_CHECK
    check_dict[SUBJECT]["GO2_CHECK"] = GO2_CHECK
    
    if not ANATS:
        ANAT_CHECK = "FAIL"
    else:
        ANAT_CHECK = "PASS"
        
    check_dict[SUBJECT]["ANAT_CHECK"] = ANAT_CHECK
    phase_check = "FAIL"
    mag1_check = "FAIL"
    mag2_check = "FAIL"
    if not FMAPS:
        print("********ERROR! NO FMAPS: ", SUBJECT)
        fmap_check = "FAIL"
    else:
        for file in FMAPS:
            name = file.split("/")[-1]
            IDENTIFIER = name.split("_")[2]
            if "phasediff" in IDENTIFIER:
                phase_check = "PASS"
                phase_file = file
            if "magnitude1" in IDENTIFIER:
                mag1_check = "PASS"
                mag1_file = file
            if "magnitude2" in IDENTIFIER: 
                mag2_check = "PASS"
                mag2_file = file
        #print(phase_check, mag1_check, mag2_check)
        if phase_check and mag1_check and mag2_check == "PASS":
            fmap_check = "PASS"
        elif phase_check and mag1_check and mag2_check != "PASS":
            fmap_check = "FAIL"
        else:
            fmap_check = "PARTIAL FAIL"
        check_dict[SUBJECT]["FMAP_CHECK"] = fmap_check

In [ ]:
df = pd.DataFrame.from_dict(check_dict, orient='index')